In [1]:
import pandas as pd
import numpy as np

In [2]:
x=pd.read_csv(r"C:\Users\Armaan\OneDrive\Desktop\WORKSPACE\Datasets\Twitter Sentiment\train.csv")
x

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...
10975,569934458364813313,neutral,American,NaN,Cottopanama85,NaN,0,@AmericanAir followback,NaN,2015-02-23 10:58:58 -0800,"ohio,panama",NaN
10976,568564006329434113,positive,United,NaN,PaulBEsteves,NaN,0,@united thanks for the help. Wish the phone re...,NaN,2015-02-19 16:13:17 -0800,Brooklyn,Eastern Time (US & Canada)
10977,569643648910028801,negative,US Airways,NaN,runfixsteve,NaN,0,@usairways the. Worst. Ever. #dca #customerser...,NaN,2015-02-22 15:43:24 -0800,"St. Augustine, Florida",NaN
10978,568864981917110272,negative,US Airways,NaN,CLChicosky,NaN,0,@nrhodes85: look! Another apology. DO NOT FLY ...,NaN,2015-02-20 12:09:15 -0800,NaN,NaN


In [3]:
def set_sentiment(text):
    if(text=="positive"):
        return 1
    else:
        return 0

In [4]:
x["airline_sentiment"]=x["airline_sentiment"].apply(set_sentiment)

In [5]:
data=[]
for i in range(x.shape[0]):
    data.append((x["text"][i],x["airline_sentiment"][i]))


In [6]:
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords ,wordnet
from nltk import WordNetLemmatizer, pos_tag, NaiveBayesClassifier
from sklearn import naive_bayes


lemmatizer=WordNetLemmatizer()

In [7]:
punctuations=string.punctuation
stops=stopwords.words('english')
stops+=punctuations
stops=set(stops)

In [8]:
def get_simple_pos(tag):
    tag=str(tag)
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADJ
    else:
        return wordnet.NOUN

In [9]:
def clean_text(text):
    output=[]
    for w in text:
        w=w.lower()
        if w not in stops:
            pos=pos_tag([w])
            clean_w=lemmatizer.lemmatize(w,pos=get_simple_pos(pos))
            output.append(clean_w)
    return output

In [10]:
data[0:3][:]

[('@SouthwestAir I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled',
  0),
 ('@SouthwestAir seeing your workers time in and time out going above and beyond is why I love flying with you guys. Thank you!',
  1),
 ('@united Flew ORD to Miami and back and  had great crew, service on both legs. THANKS',
  1)]

In [11]:
clean_data=[(clean_text(text.split()),sentiment) for text,sentiment in data]

In [12]:
clean_data[0]

(['@southwestair',
  'scheduled',
  'morning,',
  '2',
  'day',
  'fact,',
  'yes..not',
  'sure',
  'evening',
  'flight',
  'one',
  'cancelled',
  'flightled'],
 0)

In [13]:
l=len(clean_data)
training_data=clean_data[0:(int(l*0.75))]
testing_data=clean_data[(int(l*0.75)):]

In [14]:
int(l*0.75)

8235

In [15]:
testing_data[0]

(['@usairways',
  'almost',
  'home',
  '...east',
  'coast',
  'freeze',
  'headed',
  'phl',
  'connect',
  'avp',
  'tgif',
  'http://t.co/wxowzhpgfl'],
 0)

In [16]:
all_words=[]
for i in range(len(training_data)):
    all_words+=training_data[i][0]

In [17]:
all_words

['@southwestair',
 'scheduled',
 'morning,',
 '2',
 'day',
 'fact,',
 'yes..not',
 'sure',
 'evening',
 'flight',
 'one',
 'cancelled',
 'flightled',
 '@southwestair',
 'seeing',
 'worker',
 'time',
 'time',
 'going',
 'beyond',
 'love',
 'flying',
 'guys.',
 'thank',
 'you!',
 '@united',
 'flew',
 'ord',
 'miami',
 'back',
 'great',
 'crew,',
 'service',
 'legs.',
 'thanks',
 '@southwestair',
 '@dultch97',
 "that's",
 'horse',
 'radish',
 '😤🐴',
 '@united',
 'flight',
 'ord',
 'delayed',
 'air',
 'force',
 'one,',
 'last',
 'flight',
 'sbn',
 '8:20,',
 '5',
 'min',
 'landed.',
 '@united',
 'load',
 'u',
 'flying',
 'sardine',
 'knew',
 'pilot',
 '2',
 'hour',
 'late',
 'flight??',
 '#incompetent',
 'beyond',
 'belief',
 '@jetblue',
 'stock',
 'response.',
 'delay',
 'frustrating',
 'poor',
 'cust',
 'serv',
 '&amp;',
 'told',
 '3',
 'ppl',
 'wait',
 '&amp;',
 "they'd",
 'come',
 'back',
 'not.',
 '@jetblue',
 "that'd",
 'nice!',
 'hoping',
 'rack',
 'enough',
 'mile',
 'take',
 'trip',

In [18]:
freq=nltk.FreqDist(all_words)
common=freq.most_common(5000)
features=[i[0] for i in common]

In [19]:
features

['@united',
 'flight',
 '@usairways',
 '@americanair',
 '@southwestair',
 '@jetblue',
 'get',
 'cancelled',
 'hour',
 'u',
 'customer',
 'service',
 'thanks',
 'thank',
 '&amp;',
 'need',
 "i'm",
 'time',
 '2',
 'hold',
 'would',
 'still',
 'plane',
 'help',
 'bag',
 'one',
 'got',
 "can't",
 'back',
 '@virginamerica',
 'call',
 'gate',
 'late',
 'please',
 'like',
 'delayed',
 'airline',
 'day',
 'make',
 'delay',
 'trying',
 'flight.',
 'agent',
 'flightled',
 'guy',
 'phone',
 'know',
 'fly',
 'never',
 'seat',
 'waiting',
 'going',
 'change',
 'ticket',
 "i've",
 'flying',
 '3',
 '4',
 'take',
 'wait',
 'minute',
 'great',
 'even',
 'way',
 'another',
 'see',
 'check',
 'hr',
 'go',
 'could',
 'min',
 'airport',
 'last',
 'told',
 'new',
 'getting',
 'say',
 'lost',
 'due',
 'want',
 'really',
 'first',
 'people',
 'problem',
 'good',
 'next',
 'home',
 'two',
 'you.',
 'travel',
 'made',
 'let',
 'someone',
 'united',
 "that's",
 'worst',
 'love',
 'dm',
 'luggage',
 'weather',
 '

In [20]:
def count(word,dict):
    k=0
    for w in dict:
        if(word==w):
            k+=1
    return k

In [21]:
def gen_data_dict(text):
    output={}
    for i in range(len(features)):
        output[features[i]]=count(features[i],text)
    return output

In [22]:
x_train=[(gen_data_dict(text),sentiment) for text,sentiment in training_data]


In [23]:
idf=[]
for i in range(len(features)):
    dfi=0
    for j in range(len(x_train)):
        dfi+=x_train[j][0][features[i]]
    idf.append(1/dfi)
len(idf)

5000

In [24]:
len(features)

5000

In [25]:
temp=np.array(idf)
temp.min()

0.00046425255338904364

In [26]:
type(x_train[0][0])
x_train[8234]

({'@united': 1,
  'flight': 0,
  '@usairways': 0,
  '@americanair': 0,
  '@southwestair': 0,
  '@jetblue': 0,
  'get': 0,
  'cancelled': 0,
  'hour': 0,
  'u': 0,
  'customer': 0,
  'service': 0,
  'thanks': 1,
  'thank': 0,
  '&amp;': 0,
  'need': 0,
  "i'm": 0,
  'time': 0,
  '2': 0,
  'hold': 0,
  'would': 0,
  'still': 0,
  'plane': 0,
  'help': 0,
  'bag': 0,
  'one': 0,
  'got': 0,
  "can't": 0,
  'back': 0,
  '@virginamerica': 0,
  'call': 0,
  'gate': 0,
  'late': 0,
  'please': 0,
  'like': 0,
  'delayed': 0,
  'airline': 0,
  'day': 0,
  'make': 0,
  'delay': 0,
  'trying': 0,
  'flight.': 0,
  'agent': 0,
  'flightled': 0,
  'guy': 0,
  'phone': 0,
  'know': 0,
  'fly': 0,
  'never': 0,
  'seat': 0,
  'waiting': 0,
  'going': 0,
  'change': 0,
  'ticket': 0,
  "i've": 0,
  'flying': 0,
  '3': 0,
  '4': 0,
  'take': 0,
  'wait': 0,
  'minute': 0,
  'great': 0,
  'even': 0,
  'way': 0,
  'another': 0,
  'see': 0,
  'check': 0,
  'hr': 0,
  'go': 0,
  'could': 0,
  'min': 0,
  

In [27]:
index=[]
for i in range(len(idf)-1):
    l=len(index)
    if (idf[i]>0.9 or idf[i]<0.05):
        for j in range(len(x_train)):
            x_train[j][0].pop(features[i])
        index.append(i)


In [28]:
len(idf),len(features)

(5000, 5000)

In [29]:
for i in index:
    del features[i]
    del idf[i]

In [30]:
len(idf),len(features)

(4403, 4403)

In [31]:
x_test=[(gen_data_dict(text),sentiment) for text,sentiment in testing_data]

In [32]:
len(x_test[0][0])

4403

In [33]:
nbc=NaiveBayesClassifier.train(x_train)

In [34]:
nltk.classify.accuracy(nbc,x_test)

0.8451730418943534

In [36]:
from sklearn.svm import SVC
from nltk import SklearnClassifier


In [37]:
svm=SVC()
classifiersklearn=SklearnClassifier(svm)

In [38]:
classifiersklearn.train(x_train)

<SklearnClassifier(SVC())>

In [41]:
nltk.classify.accuracy(classifiersklearn,x_test)

0.8459016393442623

In [42]:
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier()

In [45]:
classifiersklearn1=SklearnClassifier(rfc)
classifiersklearn1.train(x_train)


<SklearnClassifier(RandomForestClassifier())>

In [46]:
nltk.classify.accuracy(classifiersklearn1,x_test)

0.8076502732240437

In [53]:
sentiments=[sentiment for text,sentiment in clean_data]
sentiments

[0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [48]:
x_data=[" ".join(text) for text,sentiment in clean_data]

In [56]:
x_train1,x_test1,y_train1,y_test1=train_test_split(x_data,sentiments,random_state=0)

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,


In [50]:
count_vec=CountVectorizer(max_features=3000)

In [57]:
a=count_vec.fit_transform(x_train1)
a.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [60]:
len(count_vec.get_feature_names())

c:\Users\Armaan\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


3000

In [62]:
b=count_vec.fit_transform(x_test1)

In [66]:
svm=SVC(kernel="rbf",C=10)

In [67]:
svm.fit(a,y_train1)

SVC(C=10)

In [68]:
svm.score(b,y_test1)

0.8335154826958106